# **P**rincipal **O**rthogonal **L**atent **C**omponents **A**nalysis Net (POLCA-Net)

In [1]:
import polcanet.utils
%load_ext autoreload
%autoreload 2

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [ ]:
import scienceplots
from IPython.display import display

type(scienceplots)
plt.style.use(["science", "no-latex"])

# Query the current default figure size
current_fig_size = plt.rcParams["figure.figsize"]
print(f"Current default figure size: {current_fig_size}")

# Define a scalar factor
scalar_factor = 1.5

# Multiply the current figure size by the scalar factor
new_fig_size = [size * scalar_factor for size in current_fig_size]

# Set the new default figure size
plt.rcParams["figure.figsize"] = new_fig_size

print(f"New default figure size: {new_fig_size}")

In [ ]:
import numpy as np
import torch
import torchinfo
from sklearn import datasets

In [ ]:
from polcanet import LinearDecoder, PolcaNet, utils as ut
from polcanet.aencoders import DenseEncoder

In [ ]:
import polcanet.reports as report

In [ ]:
import random

random_seed = 5
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

exp = ut.ExperimentInfoHandler(
    name="digits_dataset_8_components",
    description="POLCA-Net on digits dataset",
    random_seed=random_seed,
)
ut.set_save_fig(True)
ut.set_save_path(str(exp.get_experiment_folder()))
print(f"Saving Images: {ut.get_save_fig()}, saving in path: {ut.get_save_path()}")

### Load dataset

In [ ]:
digits = datasets.load_digits()
X = digits.data / 255
y = digits.target
images = X.reshape(X.shape[0], 8, 8)
ut.plot_train_images(images, "digits dataset images", n=10)

### Fit standard sklearn PCA

In [ ]:
n_components = 8
fig, axs = plt.subplots(1, 1, sharex=True, sharey=True, layout="constrained")
pca = ut.get_pca(X, ax=axs, title="PCA on the digits dataset", n_components=n_components)
plt.show()
Xpca = pca.transform(X)

### Fit POLCANet

In [ ]:
ae_input = X
act_fn = torch.nn.SiLU
input_dim = (ae_input.shape[1],)
latent_dim = pca.n_components

encoder = DenseEncoder(
    input_dim=input_dim,
    latent_dim=latent_dim,
    num_layers=3,
    act_fn=act_fn,
    first_layer_size=256,
    # hidden_size=512,
)

decoder = LinearDecoder(
    latent_dim=latent_dim,
    input_dim=input_dim,
    hidden_dim=256,
    num_layers=3,
    act_fn=act_fn,
    bias=True,
)

model = PolcaNet(
    encoder=encoder,
    decoder=decoder,
    latent_dim=latent_dim,
    alpha=1.0,  # ortgogonality loss
    beta=1.0,  # variance sorting loss
    gamma=0.0,  # variance reduction loss
    device="cuda",
    center=True,
    factor_scale=True,
)
print(model)
summary = torchinfo.summary(
    model,
    (1, input_dim[0]),
    dtypes=[torch.float],
    verbose=1,
    col_width=16,
    col_names=["kernel_size", "output_size", "num_params"],
    row_settings=["var_names"],
)
ut.save_text(str(model), "model.txt")
ut.save_text(str(summary), "model_summary.txt")

In [ ]:
model.to("cuda")
model.train_model(data=X, batch_size=512, num_epochs=10000, report_freq=10, lr=1e-3)

In [ ]:
model.train_model(data=X, batch_size=512, num_epochs=5000, report_freq=10, lr=1e-4)

In [ ]:
model.train_model(data=X, batch_size=512, num_epochs=5000, report_freq=10, lr=1e-5)

## Evaluate results

In [ ]:
report.analyze_reconstruction_error(model, X)

In [ ]:
latents, reconstructed = model.predict(X)

In [ ]:
# Assuming images are properly defined as before
N = 24
images_to_show = images[:N]
# Reconstruct and visualise the images using the autoencoder
_, ae_reconstructed = model.predict(X[:N])
ae_reconstructed = ae_reconstructed.reshape(images_to_show.shape)
# Reconstruct and visualize the imagaes by PCA
pca_latents = pca.transform(X[:N])
pca_reconstructed = pca.inverse_transform(pca_latents)
pca_reconstructed = pca_reconstructed.reshape(images_to_show.shape)

ut.visualise_reconstructed_images(
    [images_to_show, ae_reconstructed, pca_reconstructed],
    title_list=["Original", "POLCA-Net reconstruction", "PCA reconstruction"],
    cmap="gray",
    nrow=6,
)

In [ ]:
report.orthogonality_test_analysis(model, X)

In [ ]:
report.variance_test_analysis(model, X)

In [ ]:
report.linearity_tests_analysis(model, X)

## Polca Net vs. PCA

In [ ]:
o1 = widgets.Output()
o2 = widgets.Output()
with o1:
    _, _ = ut.plot2d_analysis(Xpca, y, title="PCA transform", legend=True)
with o2:
    _, _ = ut.plot2d_analysis(latents, y, title="POLCA-Net latent", legend=True)

layout = widgets.Layout(grid_template_columns="repeat(2, 600px)")
accordion = widgets.GridBox(children=[o1, o2], layout=layout)
display(accordion)

## Test Classification with two components on PCA vs POLCA Net

In [ ]:
_ = ut.make_classification_report(model, pca, X, y)

In [ ]:
experiment_data = {
    "digits": (
        images,
        model,
        pca,
    ),
}
_ = ut.image_metrics_table(experiment_data)